In [1]:
# health_monitoring/training/preprocess_dataset.ipynb
# PPG-DaLiA HR 추출 전처리
import sys
import os

# 현재 ipynb 기준으로 두 단계 위 경로를 sys.path에 추가
project_root = os.path.abspath(os.path.join("..", ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from health_monitoring.app.utils.preprocessing import load_features
from pathlib import Path
import numpy as np
import pickle

print("프로젝트 루트:", project_root)

프로젝트 루트: c:\Users\minsu\i-room-ai


In [2]:
Path.cwd()

WindowsPath('c:/Users/minsu/i-room-ai/health_monitoring/training')

In [3]:
# 데이터 경로
data_root = Path("C:/Users/minsu/Desktop/ppg+dalia/data/PPG_FieldStudy")

print("원본 데이터 경로:", data_root)

원본 데이터 경로: C:\Users\minsu\Desktop\ppg+dalia\data\PPG_FieldStudy


In [4]:
# S1 피험자의 데이터 확인
with open(data_root / "S1" / "S1.pkl", "rb") as f:
	data = pickle.load(f, encoding="latin1")

# 전체 키 확인
print("Top-level keys:", data.keys())

Top-level keys: dict_keys(['rpeaks', 'signal', 'label', 'activity', 'questionnaire', 'subject'])


In [ ]:
"""
** S1.pkl 구조
subject: 피험자 ID ('S1')				> 데이터 식별용
signal: 센서 신호 (wrist + chest)		> 모델 입력, HR 추출
rpeaks: ECG R-peak 인덱스 (64Hz 기준)		> HR/HRV 계산
label: ECG 기반 HR (8초 윈도우, 2초 shift)	> 모델 학습용 ground truth
activity: 활동 라벨 (0~8, 4Hz)				> 구간 분할, 행동 분석
questionnaire: 피험자 정보 (성별, 나이 등)	> 분석 보조, 그룹 비교
"""

In [ ]:
"""
** Chest 센서 (signal['chest'])
ACC: 가속도(움직임)  > 활동 감지
ECG: 심전도			> R-peak 감지, HRV 계산
EMG: 근전도			> 근육 활동 분석
EDA: 피부 전도도	> 스트레스 반응
Temp: 피부 온도		> 생리적 상태
Resp: 호흡			> 호흡률, 스트레스
"""

In [5]:
# 하위 구조 확인
print("Signal Keys (chest):", data['signal']['chest'].keys())
print("Signal keys (wrist):", data['signal']['wrist'].keys())
print("Other keys:")
for key in ['label', 'activity', 'rpeaks', 'questionnaire']:
    print(f"{key}: type={type(data[key])}, length={len(data[key]) if hasattr(data[key], '__len__') else 'N/A'}")

Signal Keys (chest): dict_keys(['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'])
Signal keys (wrist): dict_keys(['ACC', 'BVP', 'EDA', 'TEMP'])
Other keys:
label: type=<class 'numpy.ndarray'>, length=4603
activity: type=<class 'numpy.ndarray'>, length=36848
rpeaks: type=<class 'numpy.ndarray'>, length=11431
questionnaire: type=<class 'dict'>, length=6


In [ ]:
# BVP (Blood Volume Pulse) 신호: 손목 센서
bvp = data['signal']['wrist']['BVP']	# PPG 기반 심박 추출 가능

# R-peaks: ECG에서 감지된 R-peak index (심장 박동 위치)
rpeaks = data['rpeaks']	# ECG sampling rate = 64Hz 기준 인덱스

# Activity: 활동 라벨 (0~8), 4Hz 샘플링
activity = data['activity']

# Label: ECG 기반 HR (심박수), 8초 윈도우, 2초 shift
labels = data['label']	# 모델 학습용 ground truth

# ECG 기반 HRV(Heart Rate Variability; 심박 변이도) 계산
ibi = np.diff(rpeaks) / 64	# IBI (Inter-Beat Interval) in seconds
hr = 60 / ibi				# HR (Heart Rate) in bpm
sdnn = np.std(ibi)			# SDNN: 표준편차 기반 HRV
rmssd = np.sqrt(np.mean(np.square(np.diff(ibi))))	# RMSSD: HRV의 민감한 지표

In [ ]:
# 사용할 참가자 ID
subjects = [f"S{i}" for i in range(1, 16)]  # S1 ~ S15

# 모든 참가자의 데이터를 담을 리스트
all_windows = []

for subject_id in subjects:
    # 각 참가자의 데이터 경로
    pkl_path = data_root / subject_id / f"{subject_id}.pkl"
    
    if not pkl_path.exists():
        print(f"[ERROR] {subject_id} 데이터 없음, 건너뜀")
        continue

    print(f"처리 중: {subject_id}")

    # HR + ACC + Activity 기반 feature 윈도우 생성
    features = load_features(pkl_path, window_size=30, step=1)

    all_windows.append(features) # 리스트에 저장

# 전체 데이터 합치기
all_windows = np.vstack(all_windows)
print("전체 윈도우 shape:", all_windows.shape)

# 저장
np.save(output_path, all_windows)
print(f"저장 완료: {output_path}")